In [19]:
import tensorflow as tf
import numpy as np
import os

from config.config import config
from models.resnet50 import deepnn

In [20]:
def pares_tf(example_proto):
    #调用接口解析一行样本
    features = tf.parse_single_example(serialized=example_proto,
                                             features={
                                                 'image_raw': tf.FixedLenFeature([], tf.string),
                                                 'height': tf.FixedLenFeature([], tf.int64),
                                                 'width': tf.FixedLenFeature([], tf.int64),
                                                 'depth': tf.FixedLenFeature([], tf.int64),
                                                 'label': tf.FixedLenFeature([], tf.int64)
                                             })
    tf_image = tf.decode_raw(features['image_raw'], tf.uint8)
    tf_height = features['height']
    tf_width = features['width']
    tf_depth = features['depth']
    print(tf.cast(tf_depth, tf.int32))
    tf_label = tf.cast(features['label'], tf.int32)
    tf_label = tf.one_hot(tf_label, depth=190, on_value=1)
    tf_image = tf.reshape(tf_image, [64, 64, 3])
    tf_image = tf.cast(tf_image, tf.float32)
    return tf_image, tf_label


In [21]:
def load_tf_dataset(file_path):
    dataset = tf.data.TFRecordDataset(filenames=[file_path])
    dataset = dataset.map(pares_tf)
    dataset = dataset.batch(128).repeat(1)

    iterator = dataset.make_one_shot_iterator()

    next_element = iterator.get_next()

    x = tf.placeholder(dtype=tf.float32, shape=(None, 64, 64, 3), name="x")
    y_ = tf.placeholder(dtype=tf.float32, shape=(None, 190), name="y_")

    image = tf.reshape(x, shape=(-1, 64, 64, 3))
    x_temp = deepnn(x)
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        x_temp = tf.nn.dropout(x_tem, keep_prob)
    logits = tf.layers.dense(x_temp, units=classes, activation=tf.nn.softmax)
     
    cross_entropy_loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=y_, logits=logits))
    train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cross_entropy_loss)

    # 定义正确值,判断二者下表index是否相等
    correct_predict = tf.equal(tf.argmax(logits, 1), tf.argmax(y_, 1))
    # 定义如何计算准确率
    accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype=tf.float32), name="accuracy")
    # 定义初始化op
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        print("start")
        sess.run(fetches=init)
        i = 0
        try:
            while True:
                # 通过session每次从数据集中取值
                image, label = sess.run(next_element)
                sess.run(train_step, feed_dict={x: image, y_: label, keep_prob: 0.5, train_mode: True})
                if i % 100 == 0:
                    train_accuracy = sess.run(accuracy, feed_dict={x: image, y_: label, keep_prob: 1.0, train_mode: False})
                    print(i, "accuracy=", train_accuracy)
                i = i + 1
        except tf.errors.OutOfRangeError:
            print("end!")


In [23]:
file_path = os.path.join(config.dataset.trainTFRecord_list[0], 'train.tfrecords')
dataset = tf.data.TFRecordDataset(filenames=[file_path])
dataset = dataset.map(pares_tf)
dataset = dataset.batch(128).repeat(1)

iterator = dataset.make_one_shot_iterator()

next_element = iterator.get_next()

x = tf.placeholder(dtype=tf.float32, shape=(None, 64, 64, 3), name="x")
y_ = tf.placeholder(dtype=tf.float32, shape=(None, 190), name="y_")


Tensor("Cast:0", shape=(), dtype=int32)


In [25]:
image = tf.reshape(x, shape=(-1, 64, 64, 3))
x_temp = deepnn(x)
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    x_temp = tf.nn.dropout(x_tem, keep_prob)
logits = tf.layers.dense(x_temp, units=classes, activation=tf.nn.softmax)

ValueError: Variable reference/batch_normalization/gamma already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/messor/users/shinerio/nyy/ResNet50Model/models/resnet50.py", line 120, in deepnn
    def weight_variable(shape):
  File "<ipython-input-4-f7875ce7a1db>", line 14, in load_tf_dataset
    logits, keep_prob, train_mode = deepnn(x, 190)
  File "<ipython-input-5-25b5c35c646c>", line 2, in <module>
    load_tf_dataset(file_path)
